<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/5A_S2_BERT%E7%94%A8%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>BERT データセットの作成 S2 </center>

###<center>リツイートの削除</center>


#0. 作業環境の準備

##0.1. google driveのマウント

##0.2. カレントディレクトリへ移動

In [1]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


##0.3. 現在時刻の取得

In [2]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  DATETIME_now = DATETIME_now.strftime("%Y-%m-%d")
  #DATETIME_now = DATETIME_now.strftime("%Y/%m/%d %H:%M:%S")
  #print(DATETIME_now)

  return DATETIME_now

print(pull_Datetime())

2021-10-03


##option. データセットの指定

In [3]:
INPUT_CSV_v5='./Assets/Assets_Output/Tweet_Datasets/v5_forBERT.csv'

#1. データセットのロード

In [5]:
import pandas as pd
df_v0=pd.read_csv('%s' % INPUT_CSV_v5, engine='python')

# データの確認
print(f'データサイズ： {df_v0.shape}')

df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

データサイズ： (483083, 16)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,RT @det_kirigakure: #毎日龍驤\n謹賀新年、明けましておめでとうございま...,2017-12-31 15:23:39+00:00,INTJ,I,N,T,J
1,RT @nunnu_zero: 今流行りの壺をうちの子にプレゼントしてみた https://...,2017-12-31 10:42:48+00:00,INTJ,I,N,T,J
2,RT @wahiko94: 下着姿の加賀殿を描いたでござる https://t.co/rX2...,2017-12-31 10:41:44+00:00,INTJ,I,N,T,J
3,RT @whimgreen: ユニコーンちゃんに膝枕してもらいたい人生だった #アズールレー...,2017-12-30 14:01:55+00:00,INTJ,I,N,T,J
4,RT @whimgreen: スカートの中見せてくれるのかと期待して「すけべ♪」って言われた...,2017-12-30 14:01:38+00:00,INTJ,I,N,T,J


#2. データセットの加工

##2.1. 必要なデータ（文章とラベル）の抽出

In [6]:
df_v0.drop(['notes', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',  'Unnamed: 0.1.1.1'],axis=1, inplace=True)
df_v0.dropna(how='any', axis=0, inplace=True)

df=df_v0
df[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,RT @det_kirigakure: #毎日龍驤\n謹賀新年、明けましておめでとうございま...,2017-12-31 15:23:39+00:00,INTJ,I,N,T,J
1,RT @nunnu_zero: 今流行りの壺をうちの子にプレゼントしてみた https://...,2017-12-31 10:42:48+00:00,INTJ,I,N,T,J
2,RT @wahiko94: 下着姿の加賀殿を描いたでござる https://t.co/rX2...,2017-12-31 10:41:44+00:00,INTJ,I,N,T,J
3,RT @whimgreen: ユニコーンちゃんに膝枕してもらいたい人生だった #アズールレー...,2017-12-30 14:01:55+00:00,INTJ,I,N,T,J
4,RT @whimgreen: スカートの中見せてくれるのかと期待して「すけべ♪」って言われた...,2017-12-30 14:01:38+00:00,INTJ,I,N,T,J


##2.2. データのランダムサンプリング

In [7]:
import random
 #データの抽出
df=df.sample(n=10000)

##2.3. データからリツイートの投稿を除去

In [8]:
#dfXXX=df['text'].str.extract(r'RT\ @(.*)', expand=True) 
df = df[~df['text'].str.contains(r'RT\ @(.*)')]
#dfXXX = df[~df['text'].str.startswith('RT ')]
df[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
100014,デレステがローディングから進まない,2017-12-30 15:22:25+00:00,INFJ,I,N,F,J
476059,【冬休みにログインしました】,2017-12-28 07:02:11+00:00,ESFP,E,S,F,P
378189,バイオリン手に入れられなかったのはとても惜しい。,2017-12-28 06:29:35+00:00,ISTP,I,S,T,P
166478,???【私の注目選手は #三好康児 ！】\n @TAGHeuerJapan をフォロー＆Jリ...,2017-11-07 01:54:38+00:00,INFP,I,N,F,P
182904,@kon_kon2x (｡-_-｡),2017-12-21 11:22:00+00:00,INFP,I,N,F,P


#3. 記録と保存

In [ ]:
FileName=pull_Datetime()
df.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6S2_forBERT_%s.csv' % FileName)
print('./Assets/Assets_Output/Tweet_Datasets/v6S2_forBERT_%s.csv' % FileName)

./Assets/Assets_Output/Tweet_Datasets/v6S2_forBERT_2021-10-03.csv


In [ ]:
def num_Type_Index(element):
  num_Users_Type=df.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()

  print(num_Users_Type.size())
  print()

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

label_EorI
E    1583
I    8417
dtype: int64

label_NorS
N    5799
S    4201
dtype: int64

label_ForT
F    6708
T    3292
dtype: int64

label_JorP
J    3355
P    6645
dtype: int64



#Option 最大単語数の計算

In [ ]:
!pip install transformers

!pip install fugashi
!pip install ipadic

     |████████████████████████████████| 2.9 MB 5.2 MB/s 
     |████████████████████████████████| 636 kB 49.3 MB/s 
     |████████████████████████████████| 3.3 MB 44.0 MB/s 
     |████████████████████████████████| 895 kB 57.0 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 490 kB 5.2 MB/s 
     |████████████████████████████████| 13.4 MB 174 kB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=9299fe28b94e2cc28c8b2aac96f5bc2d2a9c47cd20f6ee42d3f9b376e1137b24
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [ ]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

def num_max_words(TYPE_INDEX):
  sentences= df['text'].values #sentences = df.sentence.values
  labels= df['%s' % TYPE_INDEX].values #labels = df.label.values

  # 最大単語数の確認
  max_len = []
  # 1文づつ処理
  len_sentences = len(sentences)
  for sent in tqdm(sentences):
      # Tokenizeで分割
      token_words = tokenizer.tokenize(sent)
      # 文章数を取得してリストへ格納
      max_len.append(len(token_words))
  # 最大の値を確認
  len_max=max(max_len)
  #len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
  #len_q75=int(len_q75)
  print('最大単語数: ', max(max_len))

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_max_words(element)

  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  132


  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  132


  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  132


  0%|          | 0/10000 [00:00<?, ?it/s]

最大単語数:  132
